In [1]:
import torch as th
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import scipy.io as sio

from NN import *
from th_operator import calc_grad
from utils import print_model_layers

from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import os, glob

In [2]:
# Batch size
batch_size = 10000

# Task parameters
u0 = 1 # inlet flow velocity
rho = 1000 # density
mu = 1 # viscosity

# Samples
num_points_per_step = 10000  # number of spatial points per time step
num_BC_points_per_step = 1000  # number of boundary condition points per time step
num_IC_points = 2000  # number of initial condition points

# Data boundary
## Domain
x_ini, x_f, y_ini, y_f = 0, 20, 0, 8

## Time
T = 20  # total time in seconds
Delta_t = 0.1  # time step in seconds
num_time_steps = int(T / Delta_t)  # number of time steps
time_steps = np.random.uniform(0, T, num_time_steps)

## Circle
Cx, Cy, r = 4, 4, 0.5


def generate_domain_points(num_points, x_range, y_range, time_steps, circle_center, circle_radius):
    """
    Generate spatial-temporal points within specified domain excluding a circular obstacle, for given time steps.

    Parameters:
    - num_points: Number of points to generate for each time step.
    - x_range: Tuple of (min_x, max_x) for x-coordinate range.
    - y_range: Tuple of (min_y, max_y) for y-coordinate range.
    - time_steps: Array of time steps.
    - circle_center: Tuple of (Cx, Cy), the center of the circular obstacle.
    - circle_radius: Radius of the circular obstacle.

    Returns:
    - Numpy array of spatial-temporal points with shape (num_points * len(time_steps), 3), where each row represents (x, y, t),
      excluding points inside the circular obstacle.
    """
    Cx, Cy = circle_center
    xyt_points = []

    for t in time_steps:
        for _ in range(num_points):
            while True:
                x = np.random.uniform(x_range[0], x_range[1])
                y = np.random.uniform(y_range[0], y_range[1])
                # Check if the point is outside the circular obstacle
                if (x - Cx)**2 + (y - Cy)**2 >= circle_radius**2:
                    xyt_points.append([x, y, t])
                    break
    
    domain = np.array(xyt_points, dtype=np.float32)
    idx = np.lexsort((domain[:, 1], domain[:, 0], domain[:, 2]))
    return domain[idx]

def generate_boundary_points(num_points, boundary_func, time_steps):
    xyt_points = []
    for t in time_steps:
        points = boundary_func(num_points)
        t_col = np.full((points.shape[0], 1), t)
        xyt = np.hstack((points, t_col))
        xyt_points.append(xyt)
        
    bc = np.vstack(xyt_points)
    idx = np.lexsort((bc[:, 1], bc[:, 0], bc[:, 2]))
    return bc[idx]

def circle_boundary(num_points):
    # Generate points for the circle boundary
    theta = np.random.uniform(0, 2*np.pi, num_points)
    x = Cx + r * np.cos(theta)
    y = Cy + r * np.sin(theta)
    return np.vstack((x, y)).T

def wall_boundary(num_points, x_range, y_value):
    # Generate points for wall boundaries (top or bottom)
    x = np.random.uniform(x_range[0], x_range[1], num_points)
    y = np.full(num_points, y_value)
    return np.vstack((x, y)).T

def inlet_outlet_boundary(num_points, y_range, x_value):
    # Generate points for inlet or outlet boundaries
    y = np.random.uniform(y_range[0], y_range[1], num_points)
    x = np.full(num_points, x_value)
    return np.vstack((x, y)).T

def generate_initial_conditions(num_points, x_range, y_range):
    x_initial = np.random.uniform(x_range[0], x_range[1], num_points)
    y_initial = np.random.uniform(y_range[0], y_range[1], num_points)
    t_initial = np.zeros(num_points)
    xyt_initial = np.stack((x_initial, y_initial, t_initial), axis=-1)
    return xyt_initial

def train_dataset():
    # Generating boundary points for each boundary condition and time step
    xyt_eqn = generate_domain_points(num_points_per_step, (x_ini, x_f), (y_ini, y_f), time_steps, (0, 0), 0.5)
    xyt_circle = generate_boundary_points(num_BC_points_per_step, circle_boundary, time_steps)
    xyt_w1 = generate_boundary_points(num_BC_points_per_step, lambda num_points: wall_boundary(num_points, (x_ini, x_f), y_ini), time_steps)
    xyt_w2 = generate_boundary_points(num_BC_points_per_step, lambda num_points: wall_boundary(num_points, (x_ini, x_f), y_f), time_steps)
    xyt_in = generate_boundary_points(num_BC_points_per_step, lambda num_points: inlet_outlet_boundary(num_points, (y_ini, y_f), x_ini), time_steps)
    xyt_out = generate_boundary_points(num_BC_points_per_step, lambda num_points: inlet_outlet_boundary(num_points, (y_ini, y_f), x_f), time_steps)
    xyt_initial = generate_initial_conditions(num_IC_points, (x_ini, x_f), (y_ini, y_f))

    # Combine all training points
    return {
        "eqn": xyt_eqn,
        "circle": xyt_circle,
        "w1": xyt_w1,
        "w2": xyt_w2,
        "in": xyt_in,
        "out": xyt_out,
        "initial": xyt_initial,
    }

x_train = train_dataset()
# # Save the training data
# mu_str = '{:04.1f}'.format(mu)
# file_path = f"./trainX__rho_{rho:04d}__mu_{mu_str}__.mat"
# sio.savemat(file_path, {
#     "eqn": xyt_eqn,
#     "circle": xyt_circle,
#     "w1": xyt_w1,
#     "w2": xyt_w2,
#     "in": xyt_in,
#     "out": xyt_out,
#     "initial": xyt_initial,
# })

In [3]:
print("eqn shape: ", x_train["eqn"].shape)
print("circle shape: ", x_train["circle"].shape)
print("w1 shape: ", x_train["w1"].shape)
print("w2 shape: ", x_train["w2"].shape)
print("in shape: ", x_train["in"].shape)
print("out shape: ", x_train["out"].shape)
print("initial shape: ", x_train["initial"].shape)

eqn shape:  (2000000, 3)
circle shape:  (200000, 3)
w1 shape:  (200000, 3)
w2 shape:  (200000, 3)
in shape:  (200000, 3)
out shape:  (200000, 3)
initial shape:  (50000, 3)


In [4]:
# 배치학습을 위한 데이터 로더 함수를 정의합니다.
def create_dataloader(x_data, batch_size, shuffle):
    dataset = TensorDataset(th.tensor(x_data, dtype=th.float32))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return loader

def generate_combined_batch(dataloaders_iterators, batch_size_per_loader):
    batch_parts = {}
    for key, iterator in dataloaders_iterators.items():
        try:
            # 각 DataLoader의 iterator로부터 데이터 배치를 가져옴
            data, = next(iterator)
            batch_parts[key] = data[:batch_size_per_loader]
        except StopIteration:
            # 현재 DataLoader의 데이터가 끝에 도달했을 경우, iterator를 다시 시작
            dataloaders_iterators[key] = iter(dataloaders[key])
            data, = next(dataloaders_iterators[key])
            batch_parts[key] = data[:batch_size_per_loader]
    return batch_parts

dataloaders = {key: create_dataloader(x_train[key], batch_size, False) for key in x_train}
dataloaders_iterators = {key: iter(loader) for key, loader in dataloaders.items()}

# for _ in range(200):  # 200번의 iteration으로 1 epoch을 수행
#     print("---")
#     combined_batch = generate_combined_batch(dataloaders_iterators, batch_size_per_loader=10000)
#     for k, v in combined_batch.items():
#         print(f"{k}: {v}")

In [5]:
MLP = MultiLayerPerceptronClass(
    x_dim=3, y_dim=2,
    h_dim_list=[100,100,100,100],
    actv= th.nn.Tanh(),
    p_drop=0.0,
    batch_norm=False
)
device = th.device("cuda" if th.cuda.is_available() else "cpu")
MLP = MLP.to(device)
# MLP = th.nn.DataParallel(MLP,device_ids=[0,1],output_device=0)
pinn = PINN(network=MLP, rho=rho, mu=mu)

In [6]:
class L_BFGS_B:
    """
    Optimize the PyTorch model using L-BFGS-B algorithm.
    Attributes:
        model: optimization target model.
        samples: training samples.
    """

    def __init__(self, model, dataloaders_iterators, generate_func, batch_size=10000, epochs=20):
        """
        Args:
            model: optimization target model.
            x_train: training input samples as tensors.
            y_train: training target samples as tensors.
        """
        self.model = model
        self.dataloaders_iterators = dataloaders_iterators
        self.generate_combined_batch = generate_func
        self.batch_size = batch_size
        self.epochs = epochs

        self.optimizer = optim.LBFGS(
            params = self.model.parameters(),
            lr = 1,
            max_iter = 10000,
            max_eval = 10000,
            tolerance_grad = 1e-5,
            tolerance_change = 0.5 * th.finfo(float).eps,
            history_size = 50,
            line_search_fn="strong_wolfe",
            )

    def evaluate(self):
        """
        Evaluate loss and gradients for the model.
        Returns:
            loss: the loss as a scalar tensor.
        """
        def closure():
            
            self.optimizer.zero_grad()
            
            combined_batch = self.generate_combined_batch(self.dataloaders_iterators, self.batch_size)
            for key in combined_batch:
                combined_batch[key] = combined_batch[key].to(device)
            
            outputs = self.model(combined_batch)
            PDE, BC, IC, inlet = outputs["grads"]
            
            # PDE: u_eqn, v_eqn
            # BC: uv_in, uv_w1, uv_w2, uv_circle # uv_out is not used
            # IC: u_initial, v_initial
            # inlet: uv_inlet
            
            # LOSS
            criterion = nn.MSELoss(reduction="sum")
            loss_eqn = criterion(PDE, th.zeros_like(PDE))
            loss_in = criterion(BC[0], inlet)
            loss_w1 = criterion(BC[1], th.zeros_like(BC[1]))
            loss_w2 = criterion(BC[2], th.zeros_like(BC[2]))
            loss_circle = criterion(BC[3], th.zeros_like(BC[3]))
            loss_ic = criterion(IC, th.zeros_like(IC))
            loss_out = criterion(BC[4], th.zeros_like(BC[4]))
            total_loss = (loss_eqn + loss_in + loss_w1 + loss_w2 + loss_circle + loss_ic + loss_out) / 7

            total_loss.backward()

            return total_loss
        
        return closure

    def fit(self):
        """
        Train the model using L-BFGS-B algorithm.
        Args:
            max_iter: Maximum number of iterations.
        """
        self.model.train()
        # Optimize
        for epoch in range(self.epochs):
            for iter in range(2000000//self.batch_size):
                self.optimizer.zero_grad()
                loss = self.optimizer.step(self.evaluate())
                print(f"{epoch:02d}:{iter:03d}:{loss}")   
        print('Optimization finished.')
        
    def predict(self, x):
        """
        Predict using the trained model.
        Args:
            x: Input data for prediction.
        Returns:
            predictions: Predicted values by the model.
        """
        self.model.eval()  # 모델을 평가 모드로 설정
        with th.no_grad():  # 그라디언트 계산 비활성화
            outputs = self.model(x)
        return outputs

In [7]:
lbfgs = L_BFGS_B(model=pinn, dataloaders_iterators=dataloaders_iterators, generate_func=generate_combined_batch, batch_size=batch_size, epochs=2)
lbfgs.fit()

00:000:3201.523681640625
00:001:5612.5654296875
00:002:2786.58154296875
00:003:2745.634521484375
00:004:723.1945190429688
00:005:2321.205810546875
00:006:3184.42431640625
00:007:690.8748779296875
00:008:1176.2679443359375
00:009:2110.11865234375
00:010:2089.26708984375
00:011:631.3583374023438
00:012:2262.68115234375
00:013:669.0353393554688
00:014:2129.08447265625
00:015:1847.1236572265625
00:016:2877.872314453125
00:017:871.948486328125
00:018:2008.4503173828125
00:019:717.4125366210938
00:020:2265.83251953125
00:021:710.773193359375
00:022:2253.456298828125
00:023:909.116943359375
00:024:1296.6204833984375
00:025:4790.78466796875
00:026:906.7282104492188
00:027:1914.2744140625
00:028:776.9412841796875
00:029:1736.0035400390625
00:030:2398.86572265625
00:031:1415.177978515625
00:032:1927.431396484375
00:033:1299.335693359375
00:034:2853.790771484375
00:035:734.1129150390625
00:036:1312.0858154296875
00:037:971.6281127929688
00:038:895.1232299804688
00:039:986.233642578125
00:040:722.

KeyboardInterrupt: 

In [8]:
th.save(MLP.state_dict(), "./model_state_dict.pth")

---

In [9]:
# import torch as th
# import torch.optim as optim
# import torch.nn.functional as F
# from torch.utils.data import DataLoader, TensorDataset

# import numpy as np
# import scipy.io as sio

# from NN import *
# from th_operator import calc_grad
# from utils import print_model_layers

# import matplotlib.pyplot as plt
# from matplotlib.colors import Normalize

def contour(x, y, z, title, levels=100):
    """
    Contour plot.
    Args:
        x: x-array.
        y: y-array.
        z: z-array.
        title: title string.
        levels: number of contour lines.
    """

    # get the value range
    vmin = np.min(z)
    vmax = np.max(z)

    # plot a contour
    font1 = {'family':'serif','size':20}
    plt.contour(x, y, z, colors='k', linewidths=0.2, levels=levels)
    contour_filled = plt.contourf(x, y, z, cmap='rainbow', levels=levels, norm=Normalize(vmin=vmin, vmax=vmax))

    # Add the circle patch to the current axes without altering the axes limits
    circle = plt.Circle((4, 4), 0.5, color='black')
    plt.gca().add_patch(circle)

    plt.title(title, fontdict=font1)
    plt.xlabel("X", fontdict=font1)
    plt.ylabel("Y", fontdict=font1)
    plt.tick_params(axis='both', which='major', labelsize=15)

    # Add colorbar
    cbar = plt.colorbar(contour_filled, pad=0.03, aspect=25, format='%.0e')
    cbar.mappable.set_clim(vmin, vmax)
    cbar.ax.tick_params(labelsize=15)
    


def make_gif(folder_path, title):
    frames = [Image.open(image) for image in sorted(glob.glob(folder_path))]
    frame_one = frames[0]
    frame_one.save(f"./{title}.gif", format="GIF", append_images=frames, save_all=True, duration=100, loop=1)

In [10]:
# 도메인 및 시간 설정
x_min, x_max = 0, 20
y_min, y_max = 0, 8
t_min, t_max = 0, 20  # 테스트할 시간 범위
delta_t = 0.1  # 시간 단위

# 공간 그리드 포인트 수
num_x, num_y = 100, 100 # 공간 및 시간 축에 대한 포인트 수

# 공간 및 시간 축을 위한 그리드 생성
x = np.linspace(x_min, x_max, num_x)
y = np.linspace(y_min, y_max, num_y)
X, Y = np.meshgrid(x, y)
test_shape = X.shape
time_steps = np.arange(t_min, t_max + delta_t, delta_t)

# 초기화
test_xyt = np.empty((0, 3), dtype=np.float32)

# 각 시간 스텝에 대해 x, y 그리드와 t 값을 결합
for t in time_steps:
    T = np.full(X.shape, t)
    xyt = np.stack([X.ravel(), Y.ravel(), T.ravel()], axis=-1)
    test_xyt = np.vstack([test_xyt, xyt])
    
# PyTorch 텐서로 변환 및 디바이스 할당
test_xyt = th.tensor(test_xyt, dtype=th.float32)

print(test_xyt.shape)

torch.Size([2010000, 3])


In [19]:
pred_model = predictlayer(MLP)

In [21]:
for time_steps, xyt in enumerate(test_xyt.split(10000)):
    xx = xyt[..., 0].unsqueeze(-1).reshape(test_shape).numpy()
    yy = xyt[..., 1].unsqueeze(-1).reshape(test_shape).numpy()
    xyt = th.tensor(xyt, dtype=th.float32).to(device)
    p_u_v = pred_model(xyt)
    print(p_u_v)
    break

#     p, u, v = [ p_u_v[..., i].reshape(test_shape) for i in range(p_u_v.shape[-1]) ]
#     u = u.detach().cpu().numpy()
#     v = v.detach().cpu().numpy()
#     p = p.detach().cpu().numpy()
#     # compute (u, v)
#     u = u.reshape(test_shape)
#     v = v.reshape(test_shape)
#     p = p.reshape(test_shape)

#     # plot test results
#     fig = plt.figure(figsize=(15, 6))
#     contour(xx, yy, p, f'p_{time_steps:03d}')
#     plt.tight_layout()
#     plt.savefig(f'./p/Pressure_timestep_{time_steps:03d}.png')
#     plt.close(fig)

#     fig = plt.figure(figsize=(15, 6))
#     contour(xx, yy, u, f'u_{time_steps:03d}')
#     plt.tight_layout()
#     plt.savefig(f'./u/Vx_timestep_{time_steps:03d}.png')
#     plt.close(fig)

#     fig = plt.figure(figsize=(15, 6))
#     contour(xx, yy, v, f'v_{time_steps:03d}')
#     plt.tight_layout()
#     plt.savefig(f'./v/Vy_timestep_{time_steps:03d}.png')
#     plt.close(fig)
    
# make_gif("./p/*.png", "p")
# make_gif("./u/*.png", "u")
# make_gif("./v/*.png", "v")

[tensor([nan, nan, nan,  ..., nan, nan, nan], device='cuda:0',
       grad_fn=<SelectBackward0>), tensor([nan, nan, nan,  ..., nan, nan, nan], device='cuda:0',
       grad_fn=<SelectBackward0>), tensor([nan, nan, nan,  ..., nan, nan, nan], device='cuda:0',
       grad_fn=<NegBackward0>)]


C:\Users\enomazosii\AppData\Local\Temp\ipykernel_17192\635911006.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xyt = th.tensor(xyt, dtype=th.float32).to(device)


In [18]:
class predictlayer(nn.Module):
    """
    Custom layer to compute derivatives for the steady Navier-Stokes equation using PyTorch.
    # model = Create_Model()
    # gradient_layer = GradientLayer(model)
    """
    def __init__(self, model):
        super(predictlayer, self).__init__()
        self.model = model

    def forward(self, xyt):
        x, y, t = xyt[..., 0], xyt[..., 1], xyt[..., 2]
        x.requires_grad_(True)
        y.requires_grad_(True)
        t.requires_grad_(True)
        # Combine x and y and predict u, v, p
        output = self.model(th.stack([x, y, t], dim=-1))
        psi = output[..., 0]
        p_pred = output[..., 1]
        u_pred = calc_grad(psi, y)
        v_pred = -calc_grad(psi, x)

        return [p_pred, u_pred, v_pred]